In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.regularizers import l2

### 导入宫颈癌数据集
# Импорт наборов данных
 
 

In [2]:
import pandas as pd


data = pd.read_csv('C:/Users/韩韩/Desktop/8/信息系统/宫颈癌预测/kag_risk_factors_cervical_cancer.csv')






In [3]:
data

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,?,?,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,34,3.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
854,32,2.0,19.0,1.0,0.0,0.0,0.0,1.0,8.0,0.0,...,?,?,0,0,0,0,0,0,0,0
855,25,2.0,17.0,0.0,0.0,0.0,0.0,1.0,0.08,0.0,...,?,?,0,0,0,0,0,0,1,0
856,33,2.0,24.0,2.0,0.0,0.0,0.0,1.0,0.08,0.0,...,?,?,0,0,0,0,0,0,0,0


### 缺失值处理
# Обработка пропущенных значений

In [4]:
# 将未知值?设置为空值
 
# Установить неизвестное значение? Установить значение null
 
 
data.replace('?', np.nan, inplace=True)

In [5]:
missing_values = data.isnull().sum()
print(missing_values)

Age                                     0
Number of sexual partners              26
First sexual intercourse                7
Num of pregnancies                     56
Smokes                                 13
Smokes (years)                         13
Smokes (packs/year)                    13
Hormonal Contraceptives               108
Hormonal Contraceptives (years)       108
IUD                                   117
IUD (years)                           117
STDs                                  105
STDs (number)                         105
STDs:condylomatosis                   105
STDs:cervical condylomatosis          105
STDs:vaginal condylomatosis           105
STDs:vulvo-perineal condylomatosis    105
STDs:syphilis                         105
STDs:pelvic inflammatory disease      105
STDs:genital herpes                   105
STDs:molluscum contagiosum            105
STDs:AIDS                             105
STDs:HIV                              105
STDs:Hepatitis B                  

In [6]:
# 删除缺失值过多的列
# Удалите столбцы со слишком большим количеством отсутствующих значений
drop_list = ['STDs: Time since first diagnosis', 'STDs: Time since last diagnosis']
data = data.drop(drop_list, axis=1)

In [7]:
# 缺失STDs特征的样本同样缺失许多其他特征，所以选择将其删除
# Образцы, в которых отсутствовал признак STDs, 
#также не имели многих других признаков, поэтому они были отобраны для удаления
 
data.dropna(subset=['STDs'], inplace=True)

In [8]:
# 由于是否抽烟所包含的空值样本较少，并且该特征与其他两特征相关联，所以我们选择对抽烟状态缺失的样本进行删除
#IUD与Hormonal Contraceptives特征同理

 
# Поскольку признак "курить или не курить" содержит меньше нулевых образцов и коррелирует с двумя другими признаками, мы решили удалить образцы с отсутствующим статусом курения.
# ВМС идентична признаку "Гормональные контрацептивы".
 
 
data.dropna(subset=['Smokes', 'IUD', 'Hormonal Contraceptives'], inplace=True)

In [9]:
# 检查异常值
# Проверка на наличие выбросов
missing_values_list = ['Number of sexual partners', 'First sexual intercourse', 'Num of pregnancies']
for i in missing_values_list:
    print("="*40 + f"\n{data[i].value_counts().to_string()}")

Number of sexual partners
2.0     229
3.0     181
1.0     178
4.0      65
5.0      41
6.0       7
7.0       5
8.0       4
28.0      1
9.0       1
First sexual intercourse
15.0    142
17.0    130
18.0    118
16.0     96
14.0     63
19.0     48
20.0     35
21.0     20
13.0     19
23.0      8
26.0      7
22.0      7
24.0      6
27.0      5
29.0      4
28.0      3
11.0      2
25.0      2
10.0      2
12.0      2
32.0      1
Num of pregnancies
1.0     220
2.0     203
3.0     119
4.0      71
5.0      31
6.0      17
0.0      16
7.0       5
8.0       1
11.0      1
10.0      1


In [10]:
# 众数填充
mode_values = data[missing_values_list].mode().iloc[0]
data[missing_values_list] = data[missing_values_list].fillna(mode_values)

In [11]:
# 再次检查缺失值
# Снова проверьте наличие отсутствующих значений
missing_values = data.isnull().sum()
print(missing_values)

Age                                   0
Number of sexual partners             0
First sexual intercourse              0
Num of pregnancies                    0
Smokes                                0
Smokes (years)                        0
Smokes (packs/year)                   0
Hormonal Contraceptives               0
Hormonal Contraceptives (years)       0
IUD                                   0
IUD (years)                           0
STDs                                  0
STDs (number)                         0
STDs:condylomatosis                   0
STDs:cervical condylomatosis          0
STDs:vaginal condylomatosis           0
STDs:vulvo-perineal condylomatosis    0
STDs:syphilis                         0
STDs:pelvic inflammatory disease      0
STDs:genital herpes                   0
STDs:molluscum contagiosum            0
STDs:AIDS                             0
STDs:HIV                              0
STDs:Hepatitis B                      0
STDs:HPV                              0


### 标准化

In [12]:
# 将数据集的每一列特征都转化为数值格式
# Преобразуйте каждый столбец признаков в наборе данных в числовой формат
for i in data.columns:
    data[i] = pd.to_numeric(data[i], errors='coerce')

In [13]:
data

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs:HPV,STDs: Number of diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0,0,0,0,0,0,0,0,0
2,34,1.0,15.0,1.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.00,0.0,...,0.0,0,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.00,0.0,...,0.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,34,3.0,18.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0,0,0,0,0,0,0,0,0
854,32,2.0,19.0,1.0,0.0,0.0,0.0,1.0,8.00,0.0,...,0.0,0,0,0,0,0,0,0,0,0
855,25,2.0,17.0,0.0,0.0,0.0,0.0,1.0,0.08,0.0,...,0.0,0,0,0,0,0,0,0,1,0
856,33,2.0,24.0,2.0,0.0,0.0,0.0,1.0,0.08,0.0,...,0.0,0,0,0,0,0,0,0,0,0


In [14]:
# 初始化
scaler = StandardScaler()

In [15]:
features = data.columns[:-1]
data[features] = scaler.fit_transform(data[features])

In [16]:
data

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs:HPV,STDs: Number of diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,-1.061139,0.922366,-0.732634,-0.853491,-0.408904,-0.297470,-0.200073,-1.338771,-0.611164,-0.354375,...,-0.052559,-0.291585,-0.154846,-0.091287,-0.150117,-0.163933,-0.218218,-0.324072,-0.241473,0
1,-1.404960,-0.927462,-1.085348,-0.853491,-0.408904,-0.297470,-0.200073,-1.338771,-0.611164,-0.354375,...,-0.052559,-0.291585,-0.154846,-0.091287,-0.150117,-0.163933,-0.218218,-0.324072,-0.241473,0
2,0.772570,-0.927462,-0.732634,-0.853491,-0.408904,-0.297470,-0.200073,-1.338771,-0.611164,-0.354375,...,-0.052559,-0.291585,-0.154846,-0.091287,-0.150117,-0.163933,-0.218218,-0.324072,-0.241473,0
3,2.835493,1.538975,-0.379920,1.205095,2.445561,8.555488,15.760307,0.746954,0.209213,-0.354375,...,-0.052559,-0.291585,6.458009,-0.091287,6.661456,-0.163933,-0.218218,-0.324072,-0.241473,0
4,2.147852,0.305757,1.383649,1.205095,-0.408904,-0.297470,-0.200073,0.746954,3.490723,-0.354375,...,-0.052559,-0.291585,-0.154846,-0.091287,-0.150117,-0.163933,-0.218218,-0.324072,-0.241473,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,0.772570,0.305757,0.325507,-1.539686,-0.408904,-0.297470,-0.200073,-1.338771,-0.611164,-0.354375,...,-0.052559,-0.291585,-0.154846,-0.091287,-0.150117,-0.163933,-0.218218,-0.324072,-0.241473,0
854,0.543356,-0.310852,0.678221,-0.853491,-0.408904,-0.297470,-0.200073,0.746954,1.576509,-0.354375,...,-0.052559,-0.291585,-0.154846,-0.091287,-0.150117,-0.163933,-0.218218,-0.324072,-0.241473,0
855,-0.258891,-0.310852,-0.027207,-1.539686,-0.408904,-0.297470,-0.200073,0.746954,-0.589287,-0.354375,...,-0.052559,-0.291585,-0.154846,-0.091287,-0.150117,-0.163933,-0.218218,-0.324072,4.141256,0
856,0.657963,-0.310852,2.441791,-0.167296,-0.408904,-0.297470,-0.200073,0.746954,-0.589287,-0.354375,...,-0.052559,-0.291585,-0.154846,-0.091287,-0.150117,-0.163933,-0.218218,-0.324072,-0.241473,0


# 分割数据集
# Разделите набор данных

In [17]:
X = data.drop('Biopsy', axis=1)
y = data['Biopsy']

In [18]:
y.value_counts()

Biopsy
0    676
1     50
Name: count, dtype: int64

In [19]:
# 将20%的数据用于测试
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [20]:
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Training set shape: (580, 33) (580,)
Testing set shape: (146, 33) (146,)


# 重采样，使数据平衡

# Повторная выборка для балансировки данных ###

In [21]:
sm = SMOTE(random_state=101)

In [22]:
X_train, y_train = sm.fit_resample(X_train, y_train)

In [23]:
print('Resampled dataset shape %s' % Counter(y_train))

Resampled dataset shape Counter({0: 545, 1: 545})


# 评估指数
# Индекс оценки

In [24]:
def evaluate_metrics(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    precision = precision_score(y, y_pred)
    print(f"Precision: {precision:.2f}")

    recall = recall_score(y, y_pred)
    print(f"Recall: {recall:.2f}")

    f1 = f1_score(y, y_pred)
    print(f"F1 Score: {f1:.2f}")

# 逻辑回归
# Логистическая регрессия

In [25]:
# 初始化
logis = LogisticRegression()

In [26]:
logis.fit(X_train, y_train)

LogisticRegression()

In [27]:
y_pred = logis.predict(X_test)

In [28]:
# 评估模型
evaluate_metrics(y_test, y_pred)

Accuracy: 0.95
Precision: 0.71
Recall: 0.80
F1 Score: 0.75


### SVM

In [29]:
# 初始化
svm = SVC(probability=True)

In [30]:
svm.fit(X_train, y_train)

SVC(probability=True)

In [31]:
y_pred = svm.predict(X_test)

In [32]:
# 评估模型
evaluate_metrics(y_test, y_pred)

Accuracy: 0.96
Precision: 0.91
Recall: 0.67
F1 Score: 0.77


### 随机森林

In [33]:
# 初始化
rf = RandomForestClassifier(n_estimators=100, random_state=101)

In [34]:
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=101)

In [35]:
y_pred_rf = rf.predict(X_test)

In [36]:
evaluate_metrics(y_test, y_pred)

Accuracy: 0.96
Precision: 0.91
Recall: 0.67
F1 Score: 0.77


### Keras

In [37]:
# 构建模型
Kmodel = keras.Sequential([
    layers.Dense(256, activation = 'relu', input_shape = (X_train.shape[1],)),
    layers.Dense(256, activation = 'relu'),
    layers.Dense(256, activation='relu', kernel_regularizer=l2(0.12)), # 使用l2正则避免过拟合
    layers.Dense(1, activation = 'sigmoid')
])

d:\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [38]:
# 编译模型
Kmodel.compile(optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics = ['accuracy',
                   tf.keras.metrics.Precision(),
                   tf.keras.metrics.Recall()])

In [39]:
# 训练
history = Kmodel.fit(X_train, y_train, epochs = 10, batch_size = 16, validation_split = 0.2)

Epoch 1/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8722 - loss: 22.2405 - precision: 0.8076 - recall: 0.8588 - val_accuracy: 0.9633 - val_loss: 5.2303 - val_precision: 1.0000 - val_recall: 0.9633
Epoch 2/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9392 - loss: 3.5813 - precision: 0.9083 - recall: 0.9337 - val_accuracy: 0.8991 - val_loss: 0.8981 - val_precision: 1.0000 - val_recall: 0.8991
Epoch 3/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9555 - loss: 0.5030 - precision: 0.9322 - recall: 0.9477 - val_accuracy: 0.9771 - val_loss: 0.2028 - val_precision: 1.0000 - val_recall: 0.9771
Epoch 4/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9617 - loss: 0.1530 - precision: 0.9346 - recall: 0.9636 - val_accuracy: 0.9312 - val_loss: 0.2436 - val_precision: 1.0000 - val_recall: 0.9312
Epoch 5/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9717 - loss: 0.1196 - precision: 0.9459 - recall: 0.9815 - val_accuracy: 1.0000 - val_loss: 0.1251 - va

In [40]:
loss, accuracy, precision, recall = Kmodel.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
f1_score = 2 * (precision * recall) / (precision + recall)
print(f"Test F1 Score: {f1_score}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9266 - loss: 0.3400 - precision: 0.8505 - recall: 0.4725 
Test Accuracy: 0.9452054500579834
Test Precision: 0.8181818127632141
Test Recall: 0.6000000238418579
Test F1 Score: 0.6923077062389551
